In [1]:
import numpy as np 
import pandas as pd 
import requests
import xlsxwriter 
import math 
from scipy import stats 

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW'])]          #Removing these rows as these contain errors
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2460320934837,
 'week52high': 189.21,
 'week52low': 133.7,
 'week52highSplitAdjustOnly': 188.28,
 'week52lowSplitAdjustOnly': 131.03,
 'week52change': -0.06227280397193188,
 'sharesOutstanding': 16586506224,
 'float': 0,
 'avg10Volume': 69043178,
 'avg30Volume': 92497000,
 'day200MovingAvg': 156.01,
 'day50MovingAvg': 151.7,
 'employees': 148223,
 'ttmEPS': 6.18,
 'ttmDividendRate': 0.9188649002160815,
 'dividendYield': 0.006259221185766104,
 'nextDividendDate': '',
 'exDividendDate': '2022-10-24',
 'nextEarningsDate': '2023-01-17',
 'peRatio': 25.140790418003128,
 'beta': 1.3340589635134041,
 'maxChangePercent': 59.7275566922768,
 'year5ChangePercent': 2.634766557693028,
 'year2ChangePercent': 0.3475271258106519,
 'year1ChangePercent': -0.0590543471559519,
 'ytdChangePercent': -0.14509933119771615,
 'month6ChangePercent': 0.06001495239252636,
 'month3ChangePercent': -0.09867294316846718,
 'month1ChangePercent': 0.02855932914971214,
 'day30Cha

In [4]:
data['year1ChangePercent']

-0.0590543471559519

In [ ]:
# Calling the batch API function

In [5]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

<ipython-input-6-ae397cad2ed5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-6-ae397cad2ed5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-6-ae397cad2ed5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-6-ae397cad2ed5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_dataframe = final_dataframe.append(
<ipython-input-6-ae397cad2ed5>:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conca

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,158.20,0.010199,N/A
1,AAL,14.78,-0.269215,N/A
2,AAP,151.51,-0.357501,N/A
3,AAPL,152.21,-0.060814,N/A
4,ABBV,161.41,0.402702,N/A
...,...,...,...,...
496,YUM,129.12,0.018032,N/A
497,ZBH,121.51,-0.090337,N/A
498,ZBRA,277.96,-0.574493,N/A
499,ZION,53.22,-0.226207,N/A


In [31]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

<ipython-input-31-debbf5a1b4e8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,83.70,2.320836,585
1,OXY,73.29,1.263265,668
2,MPC,128.80,1.053487,380
3,VLO,142.56,1.023852,343
4,FTI,12.33,0.943063,3975
5,MRO,31.01,0.911018,1580
6,XOM,117.72,0.879106,416
7,HES,152.00,0.813394,322
8,COP,130.82,0.812777,374
9,NOV,23.39,0.790679,2095


In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

portfolio_input()
print(portfolio_size)

Enter the value of your portfolio:2500000
2500000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,LB,83.70,2.320836,585
1,OXY,73.29,1.263265,668
2,MPC,128.80,1.053487,380
3,VLO,142.56,1.023852,343
4,FTI,12.33,0.943063,3975
5,MRO,31.01,0.911018,1580
6,XOM,117.72,0.879106,416
7,HES,152.00,0.813394,322
8,COP,130.82,0.812777,374
9,NOV,23.39,0.790679,2095


In [ ]:
# To distinguish between high quality and low quality momentum stocks we use 
#the percentile score caluclated from four different time intervals
# 1-month price return percentile
# 3-month price return percentile
# 6-month price return percentile
# 1-year price return percentile

In [10]:
hqm_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'One-Year Price Return', 
                'One-Year Return Percentile',
                'Six-Month Price Return',
                'Six-Month Return Percentile',
                'Three-Month Price Return',
                'Three-Month Return Percentile',
                'One-Month Price Return',
                'One-Month Return Percentile',
                'HQM Score'
                ]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   'N/A',
                                                   data[symbol]['stats']['year1ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month6ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month3ChangePercent'],
                                                   'N/A',
                                                   data[symbol]['stats']['month1ChangePercent'],
                                                   'N/A',
                                                   'N/A'
                                                   ], 
                                                  index = hqm_columns), 
                                        ignore_index = True)
        
hqm_dataframe.columns

<ipython-input-10-238d813c8cf9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-10-238d813c8cf9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-10-238d813c8cf9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-10-238d813c8cf9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_dataframe = hqm_dataframe.append(
<ipython-input-10-238d813c8cf9>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat inst

Index(['Ticker', 'Price', 'Number of Shares to Buy', 'One-Year Price Return',
       'One-Year Return Percentile', 'Six-Month Price Return',
       'Six-Month Return Percentile', 'Three-Month Price Return',
       'Three-Month Return Percentile', 'One-Month Price Return',
       'One-Month Return Percentile', 'HQM Score'],
      dtype='object')

In [ ]:
# we'lll caluclate momentum percentile scores for the four columns

In [19]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, 'One-Year Return Percentile'] = stats.percentileofscore(hqm_dataframe['One-Year Price Return'], hqm_dataframe.loc[row, 'One-Year Price Return'])/100
    for time_period in time_periods:
        hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])/100

# Print each percentile score to make sure it was calculated properly
for time_period in time_periods:
    print(hqm_dataframe[f'{time_period} Return Percentile'])

#Print the entire DataFrame    
hqm_dataframe

0     0.529412
1     0.215686
2     0.156863
3     0.411765
4     0.941176
5     0.921569
6     0.745098
7     0.352941
8     0.313725
9     0.039216
10    0.431373
11    0.980392
12     0.72549
13    0.137255
14    0.607843
15    0.764706
16    0.784314
17    0.901961
18    0.666667
19     0.54902
20     0.27451
21    0.823529
22    0.294118
23    0.588235
24    0.019608
25    0.392157
26    0.803922
27    0.372549
28    0.509804
29    0.196078
30    0.627451
31    0.058824
32    0.490196
33    0.960784
34    0.705882
35    0.333333
36    0.078431
37    0.686275
38    0.117647
39    0.882353
40    0.568627
41    0.254902
42         1.0
43    0.647059
44     0.45098
45    0.098039
46    0.235294
47    0.862745
48    0.843137
49    0.176471
50    0.470588
Name: One-Year Return Percentile, dtype: object
0     0.823529
1     0.137255
2     0.098039
3     0.529412
4     0.666667
5     0.647059
6          1.0
7     0.215686
8      0.45098
9     0.078431
10    0.490196
11    0.686275
12    0

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,160.33,305,0.010558,0.529412,0.240608,0.823529,0.180907,0.901961,0.199126,0.803922,N/A
1,AAL,14.56,3366,-0.270201,0.215686,-0.1435,0.137255,0.044179,0.607843,0.052878,0.196078,N/A
2,AAP,157.30,311,-0.354687,0.156863,-0.168212,0.098039,-0.247081,0.039216,-0.122967,0.058824,N/A
3,AAPL,157.97,310,-0.058706,0.411765,0.06137,0.529412,-0.096299,0.196078,0.027798,0.176471,N/A
4,ABBV,160.95,304,0.405178,0.941176,0.099314,0.666667,0.160744,0.862745,0.084077,0.313725,N/A
5,ABC,166.44,294,0.375182,0.921569,0.095872,0.647059,0.128113,0.784314,0.128706,0.509804,N/A
6,ABMD,392.33,124,0.158228,0.745098,0.523096,1.0,0.44769,1.0,0.485994,1.0,N/A
7,ABT,110.11,445,-0.137449,0.352941,-0.071026,0.215686,0.004901,0.411765,0.116117,0.431373,N/A
8,ACN,295.41,165,-0.177496,0.313725,0.049541,0.45098,-0.036267,0.313725,0.094901,0.392157,N/A
9,ADBE,342.16,143,-0.519831,0.039216,-0.176067,0.078431,-0.183718,0.098039,0.097139,0.411765,N/A


In [ ]:
#Now we find the HQM score which is the arithmetic mean of the four momentum percentile scores

In [20]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

In [28]:
hqm_dataframe.sort_values(by = 'HQM Score', ascending = False)
hqm_dataframe = hqm_dataframe[:50]

In [29]:
portfolio_input()    # Assuming the value of portfolio to be 2.5 million

Enter the value of your portfolio:2500000


In [30]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in range(0, len(hqm_dataframe['Ticker'])-1):
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / hqm_dataframe['Price'][i])
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,160.33,311,0.010558,0.529412,0.240608,0.823529,0.180907,0.901961,0.199126,0.803922,0.764706
1,AAL,14.56,3434,-0.270201,0.215686,-0.1435,0.137255,0.044179,0.607843,0.052878,0.196078,0.289216
2,AAP,157.30,317,-0.354687,0.156863,-0.168212,0.098039,-0.247081,0.039216,-0.122967,0.058824,0.088235
3,AAPL,157.97,316,-0.058706,0.411765,0.06137,0.529412,-0.096299,0.196078,0.027798,0.176471,0.328431
4,ABBV,160.95,310,0.405178,0.941176,0.099314,0.666667,0.160744,0.862745,0.084077,0.313725,0.696078
5,ABC,166.44,300,0.375182,0.921569,0.095872,0.647059,0.128113,0.784314,0.128706,0.509804,0.715686
6,ABMD,392.33,127,0.158228,0.745098,0.523096,1.0,0.44769,1.0,0.485994,1.0,0.936275
7,ABT,110.11,454,-0.137449,0.352941,-0.071026,0.215686,0.004901,0.411765,0.116117,0.431373,0.352941
8,ACN,295.41,169,-0.177496,0.313725,0.049541,0.45098,-0.036267,0.313725,0.094901,0.392157,0.367647
9,ADBE,342.16,146,-0.519831,0.039216,-0.176067,0.078431,-0.183718,0.098039,0.097139,0.411765,0.156863


In [ ]:
#Now we save our output to excel

In [24]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name='Momentum Strategy', index = False)

In [25]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [26]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

In [27]:
writer.save()